In [1]:
import re

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.metrics import pairwise_distances

In [2]:
def get_weighted_costs(file_paths):
    costs = []
    for index, cost_path in enumerate(file_paths):
        dataset, algorithm, k = re.findall(r"/.+/(.+)/(.+)-k(\d+)-", str(cost_path))[0]
        with open(cost_path, "r") as f:
            weighted_cost = float(f.read())
        costs.append({
            "dataset": dataset,
            "algorithm": algorithm,
            "k": k,
            "cost": weighted_cost,
            "file_path": str(cost_path)
        })
    return pd.DataFrame(costs)

In [3]:
# data_results_dir = Path("../data/results-2k/tower/")
# data_results_dir = Path("../data/results-2k/covertype/")
data_results_dir = Path("../data/results-2k/census/")

In [4]:
cost_paths = list(data_results_dir.glob("**/real_cost.txt"))

In [5]:
len(cost_paths)

94

In [6]:
df_cost_data = get_weighted_costs(cost_paths)

In [7]:
df_aggr_cost_data = df_cost_data.groupby(["dataset", "algorithm", "k"], as_index=False).agg(
    cost_count=("cost", "count"),
    cost_mean=("cost", "mean"),
    cost_std=("cost", "std")
)
df_aggr_cost_data

,dataset,algorithm,k,cost_count,cost_mean,cost_std
0,census,group-sampling,10,11,2.485129e+08,7.018345e+06
1,census,group-sampling,20,10,1.921408e+08,2.811315e+06
2,census,group-sampling,30,10,1.603924e+08,2.273345e+06
3,census,group-sampling,40,9,1.413188e+08,2.515538e+06
4,census,group-sampling,50,8,1.289243e+08,1.102451e+06
5,census,sensitivity-sampling,10,10,2.492438e+08,4.799339e+06
6,census,sensitivity-sampling,20,9,1.902874e+08,2.965414e+06
7,census,sensitivity-sampling,30,10,1.615043e+08,2.732427e+06
8,census,sensitivity-sampling,40,9,1.409791e+08,2.226087e+06
9,census,sensitivity-sampling,50,8,1.284792e+08,1.075695e+06


In [8]:
df = pd.pivot_table(df_aggr_cost_data, values="cost_mean", index=["algorithm"], columns=["k"])
df#.style.format(precision=0, thousands=",").highlight_min(color = 'lightgreen', axis = 0)

k,10,20,30,40,50
algorithm,,,,,
group-sampling,2.485129e+08,1.921408e+08,1.603924e+08,1.413188e+08,1.289243e+08
sensitivity-sampling,2.492438e+08,1.902874e+08,1.615043e+08,1.409791e+08,1.284792e+08


In [9]:
df = pd.pivot_table(df_aggr_cost_data, values="cost_std", index=["algorithm"], columns=["k"])
df#.style.format(precision=0, thousands=",").highlight_min(color = 'lightgreen', axis = 0)

k,10,20,30,40,50
algorithm,,,,,
group-sampling,7.018345e+06,2.811315e+06,2.273345e+06,2.515538e+06,1.102451e+06
sensitivity-sampling,4.799339e+06,2.965414e+06,2.732427e+06,2.226087e+06,1.075695e+06
